# **B92 Quantum Key Distribution Protocol**

## **Overview of Quantum Key Distribution (QKD)**  
QKD allows two parties, often referred to as **Alice** and **Bob**, to securely exchange a secret key over a quantum channel (e.g., single photons). This process involves:  
1. **Qubit Transmission** – The sender (**Alice**) encodes a secret key into qubits using **two non-orthogonal states** and transmits them.  
2. **Measurement & Filtering** – The receiver (**Bob**) measures the qubits to distinguish the states as best as possible, discarding inconclusive results.  
3. **Key Sifting** – Alice and Bob keep only the positions where Bob got a conclusive result.  
4. **Error Checking** – They publicly compare a sample of bits to estimate the error rate and detect eavesdropping.  
5. **Final Key Generation** – If the error rate is acceptable, they correct errors and perform privacy amplification to generate a secure shared key.

## **Key Exchange Process in Quantum Key Distribution**

## **Alice’s Key Preparation**

In the B92 protocol, Alice generates a random bit string and encodes each bit using **two non-orthogonal quantum states**:

- A 0 is encoded as ∣0⟩
- A 1 is encoded as ∣+⟩ = $\frac{1}{\sqrt{2}}$ (∣0⟩ + ∣1⟩) 
She sends each qubit to Bob over the quantum channel.

### **Bob’s Measurement Process**  
Bob measures each qubit using a setup that tries to rule out the state that was not sent.  
- If he can conclusively rule out one state, he deduces the bit.  
- If not, the result is inconclusive and discarded.  
Bob records the conclusive bits and their positions.

### **Basis Reconciliation**  
After transmission, Alice and Bob use a public channel to identify which measurements were conclusive. They discard inconclusive bits and keep the rest as the **raw key**.

### **Error Detection and Eavesdropping Check**  
Ideally, Alice’s and Bob’s raw keys match. Noise or interception may cause errors. They compare a random portion of their raw key to estimate the error rate. If it is too high, they suspect an eavesdropper and abort the protocol. If acceptable, they perform error correction and privacy amplification to generate the final secure key.

## **B92 Protocol States**

The B92 protocol uses these non-orthogonal states:  

**∣0⟩, ∣+⟩ = $\frac{1}{\sqrt{2}}$ (∣0⟩ + ∣1⟩)**

By encoding bits with non-orthogonal states and using only conclusive results, the B92 protocol ensures secure key distribution and allows Alice and Bob to detect eavesdropping through error checking.


## **Code Implementation**

In [30]:
from classiq import *
import numpy as np

In [31]:
# B92 uses only two non-orthogonal states: |0⟩ for bit 0, |+⟩ for bit 1.

@qfunc
def b92(message: CArray[CBool], bob_bases: CArray[CBool], qba: QArray[QBit]) -> None:
    """
    Implements the quantum part of B92 protocol.
    """
    # Alice encodes: 0 -> |0⟩, 1 -> |+⟩
    repeat(message.len,
           lambda i:
           if_(message[i]==1,
               lambda: H(qba[i])
              )
          )
    
    # Bob applies measurement: 
    # He tries to detect in the state orthogonal to Alice's states.
    # If Alice sends |0⟩, Bob measures in {|1⟩} basis => Apply X before measurement.
    # If Alice sends |+⟩, Bob measures in {|−⟩} basis => Apply H, then X.

    repeat(bob_bases.len,
           lambda i:
           if_(bob_bases[i]==0,
               lambda: X(qba[i])  # Measure in {|1⟩}
              ,
               lambda:
               (H(qba[i]), X(qba[i]))  # Measure in {|−⟩}
              )
          )

# Number of qubits
SIZE = 8

np.random.seed(3)
# Alice's message: random bits {0,1}
message = np.random.randint(0, 2, SIZE).tolist()
# Bob's measurement choice: randomly pick which orthogonal basis to measure in
bob_bases = np.random.randint(0, 2, SIZE).tolist()


@qfunc
def main(qba: Output[QArray[QBit]]) -> None:
    """
    B92 protocol quantum function.
    """
    allocate(SIZE, qba)
    b92(message, bob_bases, qba)


In [32]:
# ===============================
# Quantum Model Creation
# ===============================
# Create a quantum model from the main function
qmod = create_model(main, out_file="b92_implementation")

In [33]:
# ===============================
# Set Execution Preferences
# ===============================
from classiq.execution import ExecutionPreferences

execution_preferences = ExecutionPreferences(
    num_shots=2048
)

qmod = set_execution_preferences(qmod, execution_preferences)

In [34]:
# ===============================
# Circuit Synthesis and Execution
# ===============================
# Synthesize the quantum circuit to an executable form
qprog = synthesize(qmod)

In [35]:
# Visualize the generated quantum circuit
show(qprog)

Quantum program link: https://platform.classiq.io/circuit/2ywUZTB2gFXH7PDerI72rrMH7GO


![Description](2ywUZTB2gFXH7PDerI72rrMH7GO.jpg)

In [36]:
job = execute(qprog)

In [37]:
# ===============================
# Results Processing
# ===============================
results = job.get_sample_result().parsed_counts

## **What Happens After the Measurement? (B92 Explained)**

After Bob receives and measures the qubits, the B92 protocol proceeds classically as follows:

### **1. State Announcement**
Bob announces which qubits he successfully detected (since he can only detect one of the two non-orthogonal states).

Alice tells Bob which detections correspond to valid bits.

This ensures that only unambiguous detections contribute to the key.

### **2. Raw Key Sifting**
Alice and Bob keep only the bits where Bob's detection was conclusive.

These form the raw key.

### **3. Error Estimation**
They compare a random subset of the raw key to estimate the error rate.

A low error rate suggests no eavesdropping.

If the error rate is too high, they discard the key and restart.

### **4. Error Correction (Simulated Here)**
Any remaining errors are corrected using error-correcting codes in practice.

Here, it is assumed the key is corrected perfectly.

### **5. Privacy Amplification**
Finally, Alice and Bob apply a hash function to shorten the key and eliminate any partial information that an eavesdropper might have.


In [38]:
import numpy as np

def complete_b92_protocol(measurements_list, alice_message=None, bob_bases=None):
    """
    Complete post-processing steps for the B92 protocol.

    Args:
        measurements_list: List of classiq.interface.executor.result.SampledState objects
    """
    # 1. Define known values
    if alice_message is None:
        alice_message = np.random.randint(0, 2, SIZE).tolist()
        print("Randomly generated alice_message:", alice_message)
    elif len(alice_message) != SIZE:
        raise ValueError(f"alice_message must have length {SIZE}")

    if bob_bases is None:
        bob_bases = np.random.randint(0, 2, SIZE).tolist()
        print("Randomly generated bob_bases:", bob_bases)

    # 2. Extract Bob's measurement results
    try:
        highest_shots = -1
        highest_measurement = None

        for measurement in measurements_list:
            if measurement.shots > highest_shots:
                highest_shots = measurement.shots
                highest_measurement = measurement

        bob_measurements = highest_measurement.state['qba']

    except Exception as e:
        print(f"Error extracting measurements: {e}")
        try:
            bob_measurements = measurements_list[0].state['qba']
        except Exception:
            raise ValueError("Could not extract measurement results from the SampledState objects")

    if not isinstance(bob_measurements, list) or len(bob_measurements) != SIZE:
        raise ValueError(f"Invalid measurement data: {bob_measurements}. Expected a list of SIZE bits.")

    print("\n=== B92 Protocol Steps ===")
    print("1. QUANTUM PHASE (Already completed)")
    print("   - Alice sent states for message bits:", alice_message)
    print("   - Bob's measurement bases (0=detect |1⟩, 1=detect |−⟩):", bob_bases)
    print("   - Bob's raw measurement results:", bob_measurements)

    # 3. Conclusive detection (B92: Bob only keeps conclusive measurements)
    print("\n2. CONCLUSIVE DETECTION FILTERING")

    conclusive_positions = []
    conclusive_alice_bits = []
    conclusive_bob_bits = []

    for i in range(SIZE):
        if bob_measurements[i] == 1:
            conclusive_positions.append(i)
            # Bob infers Alice's bit:
            # If Bob detects |1⟩ => Alice sent |0⟩ (bit=0)
            # If Bob detects |−⟩ => Alice sent |+⟩ => bit=1
            if bob_bases[i] == 0:
                inferred_bit = 0
            else:
                inferred_bit = 1
            conclusive_bob_bits.append(inferred_bit)
            conclusive_alice_bits.append(alice_message[i])

    print("   - Positions with conclusive results:", conclusive_positions)
    print("   - Alice's bits at these positions:", conclusive_alice_bits)
    print("   - Bob's inferred bits:", conclusive_bob_bits)

    # 4. ERROR ESTIMATION
    if conclusive_positions:
        errors = sum(a != b for a, b in zip(conclusive_alice_bits, conclusive_bob_bits))
        error_rate = errors / len(conclusive_positions)

        print("\n3. ERROR ESTIMATION")
        print(f"   - Total conclusive results: {len(conclusive_positions)}")
        print(f"   - Errors detected: {errors}")
        print(f"   - Error rate: {error_rate:.2%}")

        # 5. ERROR CORRECTION
        if error_rate < 0.15:
            print("\n4. ERROR CORRECTION")
            print("   - Error rate acceptable, correcting errors if needed.")

            corrected_positions = [pos for pos, a, b in zip(conclusive_positions, conclusive_alice_bits, conclusive_bob_bits) if a == b]
            corrected_key = [conclusive_alice_bits[i] for i, a, b in zip(range(len(conclusive_positions)), conclusive_alice_bits, conclusive_bob_bits) if a == b]

            print("   - Positions used for final key:", corrected_positions)
            print("   - Corrected key:", corrected_key)

            # 6. PRIVACY AMPLIFICATION
            final_key = None
            if corrected_key and len(corrected_key) >= 2:
                print("\n5. PRIVACY AMPLIFICATION")
                print("   - Applying simple parity hash for final secure key.")

                final_key = []
                for i in range(0, len(corrected_key) - 1, 2):
                    final_key.append(corrected_key[i] ^ corrected_key[i + 1])

                print("   - Final secure key:", final_key)
            else:
                print("\n5. PRIVACY AMPLIFICATION")
                print("   - Not enough bits for privacy amplification.")
                final_key = []
        else:
            print("\n4. PROTOCOL ABORT")
            print("   - Error rate too high — possible eavesdropper.")
            corrected_key = []
            final_key = None
    else:
        print("\nNo conclusive detection results — cannot proceed with key generation.")
        error_rate = 1.0
        corrected_key = []
        final_key = None

    return {
        "alice_raw_key": conclusive_alice_bits,
        "bob_raw_key": conclusive_bob_bits,
        "error_rate": error_rate,
        "final_key": final_key
    }


In [39]:
# ===============================
# Run the complete B92 protocol
# ===============================
b92_results = complete_b92_protocol(list(results))

print("\nFinal B92 Protocol Results:")
print(b92_results)


Randomly generated alice_message: [1, 0, 0, 0, 0, 1, 1, 0]
Randomly generated bob_bases: [0, 0, 1, 0, 0, 0, 0, 1]

=== B92 Protocol Steps ===
1. QUANTUM PHASE (Already completed)
   - Alice sent states for message bits: [1, 0, 0, 0, 0, 1, 1, 0]
   - Bob's measurement bases (0=detect |1⟩, 1=detect |−⟩): [0, 0, 1, 0, 0, 0, 0, 1]
   - Bob's raw measurement results: [1, 1, 1, 1, 1, 1, 1, 1]

2. CONCLUSIVE DETECTION FILTERING
   - Positions with conclusive results: [0, 1, 2, 3, 4, 5, 6, 7]
   - Alice's bits at these positions: [1, 0, 0, 0, 0, 1, 1, 0]
   - Bob's inferred bits: [0, 0, 1, 0, 0, 0, 0, 1]

3. ERROR ESTIMATION
   - Total conclusive results: 8
   - Errors detected: 5
   - Error rate: 62.50%

4. PROTOCOL ABORT
   - Error rate too high — possible eavesdropper.

Final B92 Protocol Results:
{'alice_raw_key': [1, 0, 0, 0, 0, 1, 1, 0], 'bob_raw_key': [0, 0, 1, 0, 0, 0, 0, 1], 'error_rate': 0.625, 'final_key': None}


## **References**

**Note:** References [2]–[6] provide foundational background on quantum cryptography and the B92 protocol but were not all directly used in this specific implementation.

[1] **Experimental Realization of Three Quantum Key Distribution Protocols**  
[Online]. Available: [https://www.researchgate.net/publication/337670669_Experimental_Realization_of_Three_Quantum_Key_Distribution_Protocols](https://www.researchgate.net/publication/337670669_Experimental_Realization_of_Three_Quantum_Key_Distribution_Protocols)

[2] C. H. Bennett, "Quantum cryptography using any two nonorthogonal states," *Physical Review Letters*, vol. 68, no. 21, pp. 3121–3124, 1992.

[3] M. Nielsen and I. Chuang, *Quantum Computation and Quantum Information*, Cambridge University Press, 2000.

[4] N. Gisin, G. Ribordy, W. Tittel, and H. Zbinden, "Quantum cryptography," *Rev. Mod. Phys.*, vol. 74, no. 1, pp. 145–195, 2002.

[5] V. Scarani, A. Acín, G. Ribordy, and N. Gisin, "Quantum cryptography protocols robust against photon number splitting attacks for weak laser pulse implementations," *Physical Review Letters*, vol. 92, no. 5, p. 057901, 2004.

[6] S. Pirandola, U. L. Andersen, L. Banchi, et al., "Advances in quantum cryptography," *Advances in Optics and Photonics*, vol. 12, no. 4, pp. 1012–1236, 2020.
